In [58]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import pandas as pd
from os import listdir
from os.path import isfile, join

In [59]:
path_tasr = 'data/fasta/TASR'
file_class = 'data/fasta/PASTEC.classif.csv'

In [60]:
df_class = pd.read_csv(file_class, sep='\t', header=None)

In [61]:
df_class.head()

,0,1,2,3,4,5,6,7
0,FAM000002,372,.,ok,noCat,noCat,NaN,CI=NA; struct=(TermRepeats: non-termLTR: 155; ...
1,FAM000003,4291,-,ok,I,LTR,incomplete,CI=35; coding=(TE_BLRtx: Gypsy-23_ST-LTR:Class...
2,FAM000004,2239,.,ok,noCat,noCat,NaN,CI=NA; struct=(TermRepeats: non-termLTR: 172; ...
3,FAM000007,371,.,ok,II,MITE,NaN,CI=20; struct=(TElength: <700bps; TermRepeats:...
4,FAM000015,483,-,ok,II,MITE,NaN,CI=20; struct=(TElength: <700bps; TermRepeats:...


In [62]:
files = [f for f in listdir(path_tasr) if isfile(join(path_tasr, f))]
for file in files:
    if "ann" in file:
        continue
    file_path = join(path_tasr, file)
    records = []
    fasta_seqs = SeqIO.parse(file_path, "fasta")
    count = 1
    for record in fasta_seqs:
        if not "FAM" in record.id:
            records.append(record)
            continue
        family = "FAM" + record.id.split("FAM",1)[1] 
        df_res = df_class[df_class[0] == family]
        new_id = record.id
        new_class = ''
        if(len(df_res.index) > 0):
            te_type = df_res.iloc[0][5]
            classification = df_res.iloc[0][7]
            if 'ClassII' in classification:
                new_class = classification.split('ClassII')[1].split(':')[2]
            elif 'ClassI' in classification:
                new_class = classification.split('ClassI')[1].split(':')[2]
            new_id = te_type + '_' + new_class + '_' + str(count)
            count += 1
        seq = SeqRecord(record.seq, id=new_id, description=record.id)
        records.append(seq)
    new_file_path = join(path_tasr, 'ann.' + file)
    SeqIO.write(records, new_file_path, "fasta")
    print(new_file_path)

data/fasta/TASR/ann.LTRs_TASR_ClassI_families.fasta
data/fasta/TASR/ann.TIRs_TASR_ClassII_families.fasta
data/fasta/TASR/ann.TRIMs_TASR_ClassI_families.fasta
data/fasta/TASR/ann.MITEs_TASR_ClassII_families.fasta
data/fasta/TASR/ann.Helitrons_TASR_ClassII_families.fasta
data/fasta/TASR/ann.SINEs_TASR_ClassI_families.fasta
data/fasta/TASR/ann.LINEs_TASR_ClassI_families.fasta
data/fasta/TASR/ann.LARDs_TASR_ClassI_families.fasta
